**Set the kernel to "Workshop Environment" from the Jupyter Kernels.**

# Data Quality Testing with Great Expectations

## Introduction
Data quality is the foundation of reliable analytics. Poor data leads to flawed insights and decisions. This workshop explores data quality testing using the powerful Great Expectations library.

## Why Data Quality Matters
- **Trust**: Quality data builds confidence in results
- **Efficiency**: Early detection prevents downstream issues 
- **Consistency**: Ensures reliable model performance
- **Governance**: Meets regulatory requirements

## Our Approach
Using the "Bike Sharing" dataset from UCI, we'll learn how to:
- Define expectations about your data
- Validate these expectations systematically
- Document and report quality issues
- Integrate quality checks into pipelines

## Goals of this Workshop
- Design valuable data quality checks to build the foundation of your flexible (AI) pricing system
- Visually inspect expectations and keep track of expectations runs over time


## Task 1: Explore the Dataset
Let's begin by exploring the Bike Sharing dataset. Download the data, load it into a dataframe, and perform initial exploratory analysis to understand its structure and contents. This website provides detailed information about the dataset used: https://archive.ics.uci.edu/dataset/275/bike+sharing+dataset. For this workshop, we made slight alterations to the columns of the dataset (to keep it simpler) and also denormalized measures like temperature etc to make it more intuitive. 

In [1]:
# Import necessary libraries for data processing and quality testing
# - great_expectations: Our primary tool for data quality validation
# - sqlite3: To connect with our SQLite database
# - pandas: For data manipulation and analysis
import great_expectations as gx
import sqlite3
import pandas as pd

from utils import database
from utils.checker import check

metric column.standard_deviation.aggregate_fn is being registered with different metric_provider; overwriting metric_provider


In [2]:
# Initialize the database with our bike sharing dataset
# This sets up a SQLite database with the necessary tables and imports the data
database.init()

# Create a connection to our database for querying
conn = sqlite3.connect("database.db")

2025-04-10 16:29:17.059 | INFO     | utils.database:reset_database:54 - Database reset completed
2025-04-10 16:29:17.060 | INFO     | utils.database:init:142 - Initializing database to step 0: 0_spring_2011
2025-04-10 16:29:17.066 | INFO     | utils.database:apply_migration:88 - Migration 0_create_table.sql from 0_spring_2011 - Successfully applied
2025-04-10 16:29:17.095 | INFO     | utils.database:apply_migration:88 - Migration 1_bike_rental_2011_spring.sql from 0_spring_2011 - Successfully applied


In [3]:
# Set up Great Expectations context
# This creates the environment where we define and validate expectations
gx_context = gx.get_context()

# Add our SQLite database as a data source for Great Expectations
# This allows us to test data directly from the database
data_source = gx_context.data_sources.add_sqlite(
    "sample", connection_string="sqlite:///database.db"
)

In [4]:
# Define the data asset we want to validate
# An asset in Great Expectations represents a table or query result that we want to test
asset_name = "bike_rental"
database_table_name = "bike_rental"
table_data_asset = data_source.add_table_asset(
    table_name=database_table_name, name=asset_name
)

# Create a batch definition that specifies which data we want to validate
# Here we're selecting the entire table for our first season (spring 2011)
full_table_batch_definition = table_data_asset.add_batch_definition_whole_table(
    name="0_spring_2011",
)

In [5]:
# Load the data into a batch and display the first few rows
# This gives us our first look at the structure and content of the dataset
full_table_batch = full_table_batch_definition.get_batch()

full_table_batch.head().data.loc[
    :, ["season", "weekday", "temp", "casual", "registered", "total"]
]

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,season,weekday,temp,casual,registered,total
0,Spring,Monday,7.98,2,11,13
1,Spring,Monday,7.98,1,6,7
2,Spring,Monday,7.98,1,5,6
3,Spring,Monday,7.98,0,1,1
4,Spring,Monday,7.04,1,1,2


In [6]:
# Query the database directly to investigate potential data quality issues
# Here we're looking for records where the 'casual' rider count equals 2
# This helps us understand the distribution of this variable
query = """
SELECT season, weekday, temp, casual, registered, total
FROM bike_rental
WHERE casual = 2
"""

pd.read_sql_query(query, conn)

,season,weekday,temp,casual,registered,total
0,Spring,Monday,7.98,2,11,13
1,Spring,Monday,7.04,2,30,32
2,Spring,Tuesday,10.80,2,58,60
3,Spring,Wednesday,7.04,2,24,26
4,Spring,Thursday,4.22,2,106,108
...,...,...,...,...,...,...
81,Spring,Monday,17.38,2,27,29
82,Spring,Tuesday,20.20,2,8,10
83,Spring,Thursday,20.20,2,14,16
84,Spring,Friday,18.32,2,11,13


# Task 2: Set Expectations for the Spring Data

Now that you've explored the dataset, it's time to define your first expectations - the rules that your data should follow to be considered high quality.

## Basic Expectations Examples

Great Expectations provides various expectation types to validate different aspects of your data. Check out all available expectation in the gallery. https://greatexpectations.io/expectations/. Here are two examples:

1. **Column Existence**
   ```python
   # Check that specific columns exist in your dataset
   expect_column_to_exist(column="temp")

2. **Set Membership**
   ```python
   # Confirm categorical variables contain only allowed values
   expect_column_values_to_be_in_set(
       column="weathersit", 
       value_set=[1, 2, 3, 4]  # 1:Clear, 2:Cloudy, 3:Light Rain, 4:Heavy Rain
   )
   ```


In [7]:
# Task 1: Check if the 'season' column exists in the dataset
# This is a fundamental check to ensure that our data has the expected structure
# Hint: Set expectations like: 
# expectation = gx.expectations.XXX where XXX represents the expectation (see example above)

### SOLUTION_START ###
expectation = gx.expectations.ExpectColumnToExist(
    column="season",
)
### SOLUTION_END ###

# Validate the expectation
result = full_table_batch.validate(expectation, result_format="COMPLETE")
check(task=1, result=result)

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

2025-04-10 16:29:17.365 | SUCCESS  | utils.checker:check:52 - Great job! The result of the expectation is correct. Continue with the next task.


In [8]:
# Task 2: Check if the season only contains Spring
# This is a more specific check to ensure that the data is consistent with our expectations

### SOLUTION_START ###
expectation = gx.expectations.ExpectColumnValuesToBeInSet(
    column="season",
    value_set=["Spring"],
)
### SOLUTION_END ###

# Validate the expectation
result = full_table_batch.validate(expectation, result_format="COMPLETE")
print("The expectation result is: ", result["success"])
check(task=2, result=result)

Calculating Metrics:   0%|          | 0/13 [00:00<?, ?it/s]

2025-04-10 16:29:17.429 | SUCCESS  | utils.checker:check:52 - Great job! The result of the expectation is correct. Continue with the next task.


The expectation result is:  False


The expectation failed because the 'season' column contains values other than 'Spring'. Let's investigate further by querying the distinct values in the 'season' column.

In [9]:
# Investigate why the expectation failed
query = """
SELECT DISTINCT season
FROM bike_rental
"""

pd.read_sql_query(query, conn)

,season
0,Spring
1,Sprung


In [10]:
# Task 3: Fix the data quality issue
# Here we need to remove records that do not meet our expectation
# This is a critical step to ensure that our dataset is clean and reliable
# We will delete records where the season is not 'Spring'
# Hint: use a sql delete statement in the query

### SOLUTION_START ###
query = """
DELETE FROM bike_rental
WHERE season != 'Spring'
"""
### SOLUTION_END ###

# Execute the DELETE query to remove records that do not meet the expectation
with conn:
    conn.execute(query)

# Re-run the validation after fixing the data
full_table_batch = full_table_batch_definition.get_batch()
result = full_table_batch.validate(expectation, result_format="COMPLETE")
print("The expectation result is: ", result["success"])

check(task=3, result=result)

Calculating Metrics:   0%|          | 0/13 [00:00<?, ?it/s]

2025-04-10 16:29:17.556 | SUCCESS  | utils.checker:check:52 - Great job! The result of the expectation is correct. Continue with the next task.


The expectation result is:  True


But, you didn't sign up for this Workshop just to see a fancy way of doing exactly the same as your basic unit test is doing, so let's get into more complex stuff

In [11]:
# Task 4: Add another expectation for the max expected number of hourly bike data per day
# Remember that the dataset already shows aggregated hourly data in the column "total"

# Write the query to find the min and max

### SOLUTION_START ###
query = """
SELECT min(total), max(total) 
FROM bike_rental
"""
### SOLUTION_END ###

# Execute the query
print(pd.read_sql_query(query, conn))

# Set the Max to be between 0.9*Max and 1.1*Max from the query before
### SOLUTION_START ###
expectation = gx.expectations.ExpectColumnMaxToBeBetween(
    column="total",
    min_value = 0.9*638,
    max_value = 1.1*638,
)
### SOLUTION_END ###

# TODO: add validate check
# Validate the expectation
result = full_table_batch.validate(expectation, result_format="COMPLETE")
check(task=4, result=result)

   min(total)  max(total)
0           1         638


Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

2025-04-10 16:29:17.582 | ERROR    | utils.checker:check:49 - Unknown task: 4. Please provide a valid task number.


# Task 3: Adjust and Set New Expectations for the Summer Data

In [12]:
database.set_step(1)

2025-04-10 16:29:17.640 | INFO     | utils.database:set_step:221 - Processing directory: 0_spring_2011
2025-04-10 16:29:17.641 | DEBUG    | utils.database:set_step:228 - Skipping 0_create_table.sql (already applied)
2025-04-10 16:29:17.641 | DEBUG    | utils.database:set_step:228 - Skipping 1_bike_rental_2011_spring.sql (already applied)
2025-04-10 16:29:17.642 | INFO     | utils.database:set_step:221 - Processing directory: 1_summer_2011
2025-04-10 16:29:17.674 | INFO     | utils.database:apply_migration:88 - Migration 0_bike_rental_2011_summer.sql from 1_summer_2011 - Successfully applied


True

In [13]:
# Create a batch definition that specifies which data we want to validate
# Here we're selecting the entire table for our first season (spring 2011)
full_table_batch_definition = table_data_asset.add_batch_definition_whole_table(
    name="1_summer_2011",
)

# Setup for the batch
full_table_batch = full_table_batch_definition.get_batch()

In [14]:
# Investigate why the expectation failed
query = """
SELECT DISTINCT season
FROM bike_rental
"""

pd.read_sql_query(query, conn)

,season
0,Spring
1,Summer


## Excursion: Data Docs
You might have noticed that at the moment, we just ran expectations and validated them. But it would be great to define a set of expectations and validate all of them at the same time. It would be even greater, if we could somehow visualise these results over time. This subchapter addresses these issues

### Cheatsheet for GX Concepts
- ***Batch***: A slide of data you want to validate
- ***Validator***: A tool that lets you work with a batch interactively and build expectations
- ***Expectation***: A rule or assertion about your data  
- ***Expectation Suite***: A collection of expectations (think: Test suite)  
- ***Data Docs***: HTML reports showing validation results  


```text
                 +----------------+
                 |   Datasource   |
                 +----------------+
                          |
                          v
                 +---------------------+
                 |   Data Connector    |
                 +---------------------+
                          |
                          v
                 +----------------+
                 |     Batch      | <------------+
                 +----------------+              |
                          |                      |
        +-----------------+                      |
        |                                        |
        v                                        |
+----------------+        uses         +------------------------+
|  BatchRequest  |-------------------> |      Validator         |
+----------------+                    +------------------------+
                                              |
                                              v
                               +-----------------------------+
                               |     Expectation Suite       |
                               +-----------------------------+
                                              |
                                              v
                             +------------------------------+
                             |     Validation Results       |
                             +------------------------------+
                                              |
                                              v
                               +------------------------+
                               |      Data Docs         |
                               +------------------------+
                        
```

#### Notice

If you get an error like this:  
"DataContextError": Cannot add ExpectationSuite with name xxx, because it already exists   
This appears because the cell was called twice.  
To solve the issue, just give the suite a new name (for example add "_1" to it or something similar)


In [15]:
# Let's define an expectation suite
# First, we create an expectation Suite to gather all our expectations in one place
suite_name = "summer_suite"
expectation_suite = gx.ExpectationSuite(name=suite_name)

# Second, we add two expectations to the expectation suite
expectation_suite.add_expectation(gx.expectations.ExpectColumnValuesToNotBeNull(column = 'total'))
expectation_suite.add_expectation(gx.expectations.ExpectColumnValuesToNotBeNull(column = 'dteday'))

# Add the suite to the context
expectation_suite = gx_context.suites.add(expectation_suite)

In [16]:
# Let's get our batch definition (we've already defined it before)
batch_def = full_table_batch_definition

In [17]:
# Set up the validator
validation_name = "summer_validator"

validation_def = gx.ValidationDefinition(
    data = batch_def,
    suite = expectation_suite,
    name = validation_name
)

# Add it the the context
validation_def = gx_context.validation_definitions.add(validation_def)

In [18]:
# Validate the results
validation_results = validation_def.run()
print(validation_results)

# Hint: Click on open in a text editor to view the whole output

Calculating Metrics:   0%|          | 0/16 [00:00<?, ?it/s]

{
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "sample-bike_rental",
          "column": "total"
        },
        "meta": {},
        "id": "1f3456a1-bd7b-4cc6-aa05-505273e82015"
      },
      "result": {
        "element_count": 4443,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "partial_unexpected_counts": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "sample-bike_rental",
          "column": "dteday"
        },
        "meta": {},
        "id": "9f1795d9-bb63-440a-abdd-ba9b3095fcbb"
      

In [19]:
# Build the visualization file
data_doc_dir = gx_context.build_data_docs()['local_site']
print(data_doc_dir)

file:///tmp/tmpzqxa9i16/index.html


In [20]:
# TODO: move this to utils and add to container build
import os
import webbrowser
from http.server import HTTPServer, SimpleHTTPRequestHandler
import threading
from urllib.parse import urlparse

def serve_docs(path, port=8000):
    """
    Serve Great Expectations data docs on a local web server
    
    Args:
        path (str): Path to the data docs directory
        port (int): Port to serve on (default: 8000)
    """
    # Parse the URL
    parsed_url = urlparse(data_doc_dir)

    # Extract the path and remove the file name
    directory_path = os.path.dirname(parsed_url.path)

    # Change to the data docs directory
    os.chdir(directory_path)
    
    # Create server
    server = HTTPServer(('localhost', port), SimpleHTTPRequestHandler)
    
    # Start server in a separate thread
    thread = threading.Thread(target=server.serve_forever)
    thread.daemon = True  # This makes the thread exit when the main program exits
    thread.start()
    
    url = f"http://localhost:{port}"
    print(f"Serving data docs at {url}")
    
    # Open browser automatically
    webbrowser.open(url)
    
    return server

In [21]:
# Serve the .html in the browser
# TODO: need to properly end and reiterate the service when function is called twice
# serve_docs(data_doc_dir)

#### Notice

If you get an error like this:  
"OSError: [Errno 98] Address already in use"
This appears because the cell was called twice.  
To solve the issue, restart the kernel and run the notebook again

Inspect the Webpage that just opened! You can click the validation with the given expectation suite (here the summer_suite) and look into all the validations that were run when calling the validator. Take some time playing around with the webpage and explore!

## More Summer Expectations
Equiped with this knowledge, let's set up the summer expectations

In [22]:
# Initialise a expectations array, which contains all our expectations and can then be added to our suite
expectations = []

In [23]:
# Task 5: First, add the expectation that the expected column values can now contain either "Spring" or "Summer"
# Hint: Look at Task 1 and follow the pattern.

### SOLUTION_START ###
expectation = gx.expectations.ExpectColumnValuesToBeInSet(
    column="season",
    value_set=["Spring", "Summer"],
)
### SOLUTION_END ###

expectations.append(expectation)

# TODO: Generate the task validation check
# Validate the expectation (to see if the task was done correctly)
# result = full_table_batch.validate(expectation, result_format="COMPLETE")
# check(task=5, result=result)

In [24]:
# Task 6: Copy Paste the Expectation set in Task 4 and add it to our expectations array.
# We want to check if the expectations hold up for summer rush.

# Copy Paste Solution from Task 4
### SOLUTION_START ###
expectation = gx.expectations.ExpectColumnMaxToBeBetween(
    column="total",
    min_value = 0.9*638,
    max_value = 1.1*638,
)
### SOLUTION_END ###

expectations.append(expectation)

In [25]:
# Add the expectations to our suite
for exp in expectations:
    expectation_suite.add_expectation(exp)

In [26]:
# Set up the validator
validation_name = "summer_validator_updated"

# Define the validator
### SOLUTION_START ###
validation_def = gx.ValidationDefinition(
    data = batch_def,
    suite = expectation_suite,
    name = validation_name
)
### SOLUTION_END ###

# Add it the the context
validation_def = gx_context.validation_definitions.add(validation_def)

In [27]:
# Validate the results by running the validator
### SOLUTION_START ###
validation_results = validation_def.run()
print(validation_results)
### SOLUTION_END ###

# Hint: Click on open in a text editor to view the whole output

Calculating Metrics:   0%|          | 0/31 [00:00<?, ?it/s]

{
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "sample-bike_rental",
          "column": "total"
        },
        "meta": {},
        "id": "1f3456a1-bd7b-4cc6-aa05-505273e82015"
      },
      "result": {
        "element_count": 4443,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": [],
        "partial_unexpected_counts": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_max_to_be_between",
        "kwargs": {
          "batch_id": "sample-bike_rental",
          "column": "total",
          "min_value": 574.2,
          "max_value": 701.8000000000001
        },
        "met

In [28]:
# Build the visualization file
data_doc_dir = gx_context.build_data_docs()['local_site']
print(data_doc_dir)

file:///tmp/tmpzqxa9i16/index.html


In [29]:
# Serve the .html in the browser
# TODO: need to be navigated back to our directory, if you leave this in, then database.set_step can not find the folder data
# serve_docs(data_doc_dir)

# Task 4: Adjust for Autumn 

In [30]:
database.set_step(2)

2025-04-10 16:29:18.401 | INFO     | utils.database:set_step:221 - Processing directory: 0_spring_2011
2025-04-10 16:29:18.402 | DEBUG    | utils.database:set_step:228 - Skipping 0_create_table.sql (already applied)
2025-04-10 16:29:18.402 | DEBUG    | utils.database:set_step:228 - Skipping 1_bike_rental_2011_spring.sql (already applied)
2025-04-10 16:29:18.403 | INFO     | utils.database:set_step:221 - Processing directory: 1_summer_2011
2025-04-10 16:29:18.403 | DEBUG    | utils.database:set_step:228 - Skipping 0_bike_rental_2011_summer.sql (already applied)
2025-04-10 16:29:18.404 | INFO     | utils.database:set_step:221 - Processing directory: 2_fall_2011
2025-04-10 16:29:18.432 | INFO     | utils.database:apply_migration:88 - Migration 0_bike_rental_2011_fall.sql from 2_fall_2011 - Successfully applied


True

In [31]:
# Create a batch definition that specifies which data we want to validate
# Here we're selecting the entire table for our first season (spring 2011)
full_table_batch_definition = table_data_asset.add_batch_definition_whole_table(
    name="2_fall_2011",
)

# Setup for the batch
full_table_batch = full_table_batch_definition.get_batch()

In [32]:
# Task 7: Let's look at the bike rental trends! 
# Calculate the current mean of total bike rentals per ***month*** 
# Hint: the mean can be calculated using avg() in SQL

### SOLUTION_START ###
query = """
SELECT mnth, avg(total)
FROM bike_rental
Group By  mnth
"""
### SOLUTION_END ###

# Execute the query
print(pd.read_sql_query(query, conn))

   mnth  avg(total)
0     3   87.842308
1     4  131.947149
2     5  182.555108
3     6  199.322222
4     7  189.974462
5     8  186.991792
6     9  177.709902
7    10  166.232840
8    11  142.095967
9    12  135.277083


#### Batch Definitions 
We will now introduce the concept of batch definitions

In [33]:
suite_name = "fall_suite"
expectation_suite = gx.ExpectationSuite(name=suite_name)

# Second, we add two expectations to the expectation suite
expectation_suite.add_expectation(gx.expectations.ExpectColumnValuesToNotBeNull(column = 'total'))
expectation_suite.add_expectation(gx.expectations.ExpectColumnValuesToNotBeNull(column = 'dteday'))

# Add the suite to the context
expectation_suite = gx_context.suites.add(expectation_suite)

In [34]:
# Now let's define an expected mean per month

# Fill in the mean of october
### SOLUTION_START ###
mean_october = 166
### SOLUTION_END ###

# We set the expectation of the value range of the mean for october
suite_name = "october_expectation_suite"

# Create the expectation for october
expectation_october = gx.expectations.ExpectColumnMeanToBeBetween(
    column='total',
    min_value=0.8*mean_october,
    max_value=1.2*mean_october
)

# Create the suite and add the expectation
expectation_suite = gx.ExpectationSuite(name=suite_name)
expectation_suite.add_expectation(expectation_october)

# Add the suite to the context
expectation_suite = gx_context.suites.add(expectation_suite)


In [ ]:
# This expectations needs to be applied to october only
# We now introduce the concept of batch definitions
# TODO: fix this, the data is not filtered correctly at the moment
batch_definition_october = table_data_asset.add_batch_definition(
    name="october_data",
    batch_kwargs={
        "query": "SELECT * FROM bike_rental WHERE mnth = 10"
    }
)

# Retrieve the batch for October
batch_october = batch_definition_october.get_batch()

TypeError: _SQLAsset.add_batch_definition() got an unexpected keyword argument 'batch_kwargs'

In [ ]:
# Define the batch
batch_def_october = gx_context.get_batch_definition(
    batch_kwargs={"data": batch_october, "column_list": ["total"]},
    expectation_suite=expectation_suite
)

AttributeError: 'Batch' object has no attribute 'filter'

In [ ]:
# Setup the validator 
validation_name = "october_validation"
validation_def = gx.ValidationDefinition(
    data=batch_def_october,
    suite=expectation_suite,
    name=validation_name
)

validation_def = gx_context.validation_definitions.add(validation_def)

In [ ]:
# Run the validation
validation_results = validation_def.run()

# Build the visualization file
data_doc_dir = gx_context.build_data_docs()['local_site']

# Serve the .html in the browser
# TODO: need to be navigated bac to our directory, if you leave this in, then database.set_step can not find the folder data
# serve_docs(data_doc_dir)

# Task 5: Check with Winter and set final expectations
You can check out all kinds of expectations here: https://greatexpectations.io/expectations/

In [ ]:
# TODO: load new dataset

In [ ]:
# TODO: check the expectations

In [ ]:
# TODO: fix what needs fixing

In [ ]:
# TODO: Maybe add something even more complex?

In [ ]:
# TODO: Recheck the Expectations

# Task 6: Verify your data and see if something shifts the next year

In [ ]:
# TODO: load new dataset

In [ ]:
# TODO: check the expectations

Discuss these expectations => did you do a good job? What changed? Do you now have confidence in your data foundation for your AI model? Discuss pros and cons of using a Testing Framework!

# Task 7: Think about AI Implementation

Could you now implement AI to design a flexible pricing model? How would you do it? What is the advantage over doing this by hand?

In [ ]:
# TODO: Make this last part better and more to the point ^^

# Cleanup

In [ ]:
# To stop the server when you're done (run this in another cell):
# TODO: add function cleanup to utils which calls server.shutdown and cleans other things up if necessary
# cleanup()
# server.shutdown()